In [118]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import statsmodels.api as sm

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Imports
Import modules utilised, specifically **Pandas** for data handling and **Plotly** for interactive data visualisation. 

I also have defined options within Plotly that allow for the graphs to be saved and manipulated locally for portability.

In [2]:
df = pd.read_csv("TeslaProduction-BusinessProductivity.csv")

In [3]:
df.shape

(23, 3)

## Data Shape
Showing 23 instances of of data in each dataset, of which there are 3 in total.

In [4]:
df.describe()

,Tesla Production,Real Output per Hour
count,23.000000,23.000000
mean,13925.130435,106.461478
std,9136.266914,1.573489
min,321.000000,104.059000
25%,6674.500000,105.108500
50%,11507.000000,106.668000
75%,22100.000000,107.523500
max,29980.000000,109.050000


## Quick Descriptive Statistics
No need to define these variables further, as they're simplistic enough:
* Few in number, no need to sample
* Scalar in values, not categorical or ordinal

Even though we don't need to sample, best to remember that this is still assumed to be a representative sample of the true distributions of each individual dataset.

In [5]:
df.head()

,Date,Tesla Production,Real Output per Hour
0,2012Q3,321,104.503
1,2012Q4,2400,104.059
2,2013Q1,4900,104.304
3,2013Q2,5150,104.147
4,2013Q3,5500,104.570


In [6]:
df.tail()

,Date,Tesla Production,Real Output per Hour
18,2017Q1,25000,107.683
19,2017Q2,22000,108.142
20,2017Q3,26150,108.849
21,2017Q4,29870,108.943
22,2018Q1,29980,109.050


In [7]:
y0 = df['Tesla Production']
y1 = df['Real Output per Hour']

trace0 = go.Box(
    y=y0,
    name = 'Tesla Production',
    marker = dict(
        color = 'rgb(214, 12, 140)',
        )
    )
trace1 = go.Box(
    y=y1,
    name = 'Real Output per Hour',
    marker = dict(
        color = 'rgb(0, 128, 128)',
        )
    )

data = [trace0, trace1]
iplot(data)

In [110]:
fig0 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='box', 
    showlegend=False,
    colors = ('#ff0074'),
    )
fig1 = df.iplot(
    columns=['Tesla Production', 'Real Output per Hour'],
    asFigure=True,
    kind='box', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Box Plots',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
)

fig1['data'].extend(fig0['data'])

iplot(fig1)

In [79]:
fig0 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    title='Real Output per Hour',
    yTitle='Real Productivity per Hour ($)',
    x='Date'
    )

iplot(fig0)

In [82]:
fig1 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='scatter', 
    showlegend=False,
    title='Teslas Produced',
    yTitle='Car Units Produced',
    colors = ('#1b073a'),
    x='Date'
    )

iplot(fig1)

In [109]:
fig2 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    yTitle='Real Output per Hour',
    x='Date',    
    )
fig3 = df.iplot(
    asFigure=True,
    kind='scatter', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Time Series',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    x='Date'
    )

fig3['data'].extend(fig2['data'])

iplot(fig3)

In [117]:
fig2 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    yTitle='Real Output per Hour',
    x='Date',
    mode='markers',
    symbol='diamond'
    )
fig3 = df.iplot(
    asFigure=True,
    kind='scatter', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Time Series',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    x='Date',
    mode='markers',
    )

fig3['data'].extend(fig2['data'])

iplot(fig3)

In [119]:
model = sm.OLS(df['Tesla Production'],df['Real Output per Hour'])

In [122]:
results = model.fit()

In [125]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Tesla Production   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     56.64
Date:                Sun, 24 Jun 2018   Prob (F-statistic):           1.60e-07
Time:                        13:56:11   Log-Likelihood:                -241.41
No. Observations:                  23   AIC:                             484.8
Df Residuals:                      22   BIC:                             485.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Real Output per Hour   131.8811 

In [133]:
df['Fitted Values']=results.fittedvalues

In [130]:
df['Residuals']=results.resid

In [134]:
df

,Date,Tesla Production,Real Output per Hour,Residuals,Fitted Values
0,2012Q3,321,104.503,-13460.970852,13781.970852
1,2012Q4,2400,104.059,-11323.415643,13723.415643
2,2013Q1,4900,104.304,-8855.726513,13755.726513
3,2013Q2,5150,104.147,-8585.021180,13735.021180
4,2013Q3,5500,104.570,-8290.806886,13790.806886
5,2013Q4,6892,105.634,-7039.128379,13931.128379
6,2014Q1,6457,104.782,-7361.765680,13818.765680
7,2014Q2,7579,105.435,-6325.884040,13904.884040
8,2014Q3,7785,106.530,-6264.293847,14049.293847
9,2014Q4,9834,105.977,-4142.363597,13976.363597


In [18]:
df['Tesla Production'].sum()

320278

In [ ]:
df['Tesla Production'].corr(df['Real Output per Hour'])

# TODO 
* Cacluate line of best fit (OLS), add to df
* distribution
* R 
* adj. R 
* Pearsons Correlation 
* heteroskedasticity robust 
* Skewness
* Kurtosis
* ones and talk about areas to move on with this stuff